In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from PIL import Image
from tqdm import tqdm

#import mc_classification_learning_extension as mc

In [2]:
np.random.seed(0)

In [3]:
class PseudoLabeledDataset(torch.utils.data.Dataset):
    def __init__(self, labeled_ds, unlabeled_ds, pseudo_labels):
        self.labeled = labeled_ds
        self.unlabeled = unlabeled_ds
        self.pseudo_labels = pseudo_labels
    
    def __len__(self):
        return len(self.labeled) + len(self.unlabeled)
    
    def __getitem__(self, index):
        if index < len(self.labeled):
            labeled_data = self.labeled[index]
            return labeled_data[0], labeled_data[1]  
        else:
            pseudo_index = index - len(self.labeled)
            unlabeled_data = self.unlabeled[pseudo_index][0]
            pseudo_label = int(self.pseudo_labels[pseudo_index])
            return unlabeled_data, pseudo_label
    

In [4]:
class FineTunedResNet():
    def __init__(self, num_classes, dropout_rate=0, criterion=nn.CrossEntropyLoss(), optimizer=optim.Adam, lr=0.0005, pretrained_lr = 0.000005, betas=(0.9, 0.999), eps=1e-8, weight_decay=0) -> None:
        self.is_finetuned = False
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.n_classes = num_classes
        self.model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

        # initially, all layers are frozen
        for param in self.model.parameters():
            param.requires_grad = False
        # only the final layer is unfrozen
        for param in self.model.fc.parameters():
            param.requires_grad = True

        num_ftrs = self.model.fc.in_features

        # Replace the final layer with a Sequential containing the Dropout and Linear layers
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(num_ftrs, self.n_classes)
        )

        self.model = self.model.to(self.device)

        self.criterion = criterion

        # define an optimizer for each layer
        self.optimizers = []
        for i in range(4):
            params = [{'params': getattr(self.model, f'layer{i+1}').parameters(), 'lr': pretrained_lr}]
            self.optimizers.append(optimizer(params, betas=betas, eps=eps, weight_decay=weight_decay))
        self.optimizers.append(optimizer([{'params': self.model.fc.parameters(), 'lr': lr}], betas=betas, eps=eps, weight_decay=weight_decay))
        
        self.schedulers = []
        # define a scheduler for each optimizer
        for opt in self.optimizers:
            self.schedulers.append(torch.optim.lr_scheduler.StepLR(opt, step_size=3, gamma=0.8))

    def alpha_weight(self, step, num_epochs, af=2):
        if step < num_epochs/2:
            return 0.0
        #elif step > num_epochs * 3/4:
        #    return af
        else:
            return ((step-num_epochs/2) / (num_epochs*3/4 - num_epochs/2))# * af

    def train(self, train_loader, n_batches, save_model=True, safepath="", pseudolabeling={"on":False, "unlabeled":[], "confidence":0.9}):
        if pseudolabeling["on"] and not pseudolabeling["unlabeled"]:
                raise Exception("Unlabeled data for pseudo labeling is missing.")
                
        self.is_finetuned = True
        
        #self.model.train()  # Set the model to training mode
        
        #if save_model and not safepath:
        #    safepath = f"ft_model_{self.n_classes}_{num_epochs}.pt"
        
        epoch = 0
        unlabeled_loader = pseudolabeling["unlabeled"]
        num_epochs = len(unlabeled_loader)/n_batches
        for j, (x_un, _) in enumerate(unlabeled_loader):
            x_un = x_un.to(self.device)
            self.model.eval()
            pred = self.model(x_un)
            _, pseudo_label = torch.max(pred, 1)
            self.model.train()
            
            out = self.model(x_un)
            u_loss = self.alpha_weight(epoch, num_epochs) * self.criterion(out, pseudo_label)
            
            u_loss.backward()
            for optimizer in self.optimizers:
                    optimizer.step()
            
            #for epoch in range(num_epochs):
        
            if j % n_batches == 0:

                # decide which layers to unfreeze in this epoch
                if epoch < len(self.optimizers):
                    for param in self.optimizers[epoch].param_groups[0]['params']:
                        param.requires_grad = True

                r_loss = 0.0
                data_loader = train_loader

                for i, data in tqdm(enumerate(data_loader, 0)):
                    x, y = data
                    x = x.to(self.device)
                    y = y.to(self.device)

                    # clear and update all optimizers
                    for optimizer in self.optimizers:
                        optimizer.zero_grad()

                    out = self.model(x)
                    loss = self.criterion(out, y)

                    loss.backward()
                    for optimizer in self.optimizers:
                        optimizer.step()

                    r_loss += loss.item()

                # update all schedulers
                for scheduler in self.schedulers:
                    scheduler.step()

                print(f"Epoch {epoch}: loss {r_loss}")
                epoch += 1
                if epoch >= num_epochs:
                    break

        #if save_model:
        #    torch.save(self.model.state_dict(), safepath)


    def validate(self, val_loader):
        if not self.is_finetuned:
            raise Exception("The model has not been fine-tuned. Call model.train first to fine-tune the model.")
            
        self.model.eval()  # Set the model to evaluation mode
        
        correct = [0] * self.n_classes
        total = [0] * self.n_classes
        with torch.no_grad():
            for i, data in enumerate(val_loader, 0):
                x, y = data
                x = x.to(self.device)
                y = y.to(self.device)
                c = []
                output = self.model(x)
                for out in output:
                    c.append(out.argmax())
                for i in range(len(y)):
                    label = y[i]
                    if label == c[i]:
                        correct[label] += 1
                    total[label] += 1
        return self.get_accuracy(correct, total)

    def test_single(self, test_img):
        if not self.is_finetuned:
            raise Exception("The model has not been fine-tuned. Call model.train first to fine-tune the model.")
        self.model.eval()
        img_tensor = test_img.unsqueeze(0)
        img_tensor = img_tensor.to(self.device)

        out = self.model(img_tensor)
        pred_class = out.argmax().item()

        return pred_class
    
    def get_accuracy(self, correct, total):
        acc_class = np.zeros((self.n_classes, 1))
        for i in range(self.n_classes):
            acc_class[i] = correct[i]/total[i]
        acc = sum(correct)/sum(total)
        return acc_class, acc
    
    def generate_pseudo_labels(self, unlabeled_loader, confidence):
            pseudo_labels = []
            high_conf_indices = []
            with torch.no_grad():
                for i, (x, _) in enumerate(unlabeled_loader):
                    x = x.to(self.device)
                    c = []
                    output = self.model(x)
                    for j in range(len(output)):
                        pred = output[j].argmax()
                        max_conf = torch.nn.functional.softmax(output[j], dim=0).max()
                        if max_conf >= confidence:
                            c.append(pred)
                            high_conf_indices.append(i * unlabeled_loader.batch_size + j)

                    pseudo_labels.extend(c)
            return pseudo_labels, high_conf_indices

In [5]:
traindir = "data/mc_data/mc_training"
validdir = "data/mc_data/mc_validation"
testdir = "data/mc_data/mc_test"

transform  = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
    [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
    )
])

train_set = datasets.ImageFolder(traindir, transform)
val_set = datasets.ImageFolder(validdir, transform)
test_set = datasets.ImageFolder(testdir, transform)

In [13]:
labeled_split = 0.1
num_data = len(train_set)
num_labeled = int(num_data * labeled_split)

# Get the set of all unique classes
classes = set(train_set.targets)
labeled_indices = []
class_labeled_indices = []
for class_idx in classes:
    class_indices = [i for i, target in enumerate(train_set.targets) if target == class_idx]
    subset_size = max(1, int(len(class_indices) * labeled_split))
    class_subset_indices = np.random.choice(class_indices, subset_size, replace=False)

    labeled_indices.extend(class_subset_indices)
    class_labeled_indices.append(class_subset_indices)

#labeled_indices = np.random.choice(num_data, num_labeled, replace=False)
unlabeled_indices = np.delete(np.arange(0, num_data, 1), labeled_indices)

In [14]:
labeled_subset = torch.utils.data.Subset(train_set, labeled_indices)
unlabeled_subset = torch.utils.data.Subset(train_set, unlabeled_indices)

In [15]:
labeled_loader = torch.utils.data.DataLoader(
    labeled_subset,
    batch_size=4,
    shuffle=True,
    num_workers=2
)

unlabeled_loader = torch.utils.data.DataLoader(
    unlabeled_subset,
    batch_size=4,
    shuffle=False,
    num_workers=2
)

classes = train_set.classes

In [9]:
val_loader = torch.utils.data.DataLoader(
        val_set,
        batch_size=4,
        shuffle=True,
        num_workers=2
    )
test_loader = torch.utils.data.DataLoader(
        test_set,
        batch_size=4,
        shuffle=True,
        num_workers=2
    )

In [16]:
ps_model = FineTunedResNet(37)
ps_settings = {"on": True, 
               "unlabeled": unlabeled_loader}
# middle number != num_epochs, 
# determines after how many batches of unlabeled data one epoch of training is done on the labeled set
ps_model.train(labeled_loader, 5, pseudolabeling=ps_settings)

73it [00:02, 25.42it/s]


Epoch 0: loss 259.48752093315125


73it [00:03, 23.86it/s]


Epoch 1: loss 198.3794651031494


73it [00:03, 20.89it/s]


...

Epoch 130: loss 8.005762005224824


73it [00:04, 17.88it/s]


Epoch 131: loss 7.1398048512637615


73it [00:04, 17.86it/s]


Epoch 132: loss 7.655453495681286


In [17]:
ps_model.validate(val_loader)

(array([[0.75      ],
        [0.55      ],
        [0.7       ],
        [0.89473684],
        [0.8       ],
        [0.77777778],
        [0.8       ],
        [0.6       ],
        [0.8       ],
        [0.65      ],
        [0.78947368],
        [1.        ],
        [0.8       ],
        [0.8       ],
        [0.9       ],
        [0.75      ],
        [0.7       ],
        [0.55      ],
        [0.89473684],
        [1.        ],
        [0.85      ],
        [1.        ],
        [0.95      ],
        [1.        ],
        [1.        ],
        [0.9       ],
        [0.9       ],
        [0.89473684],
        [0.9       ],
        [0.9       ],
        [0.95      ],
        [0.85      ],
        [0.75      ],
        [1.        ],
        [0.6       ],
        [0.95      ],
        [0.95      ]]),
 0.8337874659400545)

In [18]:
ps_model.validate(test_loader)

(array([[0.6122449 ],
        [0.72      ],
        [0.7       ],
        [0.72727273],
        [0.61      ],
        [0.71134021],
        [0.77      ],
        [0.57      ],
        [0.57      ],
        [0.61      ],
        [0.66      ],
        [0.74      ],
        [0.73      ],
        [0.56      ],
        [0.97      ],
        [0.79      ],
        [0.94949495],
        [0.68      ],
        [0.88      ],
        [0.88      ],
        [0.99      ],
        [0.95      ],
        [0.96      ],
        [1.        ],
        [0.98989899],
        [0.94      ],
        [0.91      ],
        [0.86      ],
        [0.79      ],
        [0.96      ],
        [0.97      ],
        [0.98      ],
        [0.92929293],
        [0.95      ],
        [0.61797753],
        [0.94      ],
        [0.92      ]]),
 0.8143908421913328)